In [ ]:
#topic based clustering-abstrative summary 
# based on Parrot project

In [1]:
pip install keybert

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import keybert
from transformers import pipeline
import tqdm as notebook_tqdm
import pandas as pd
import math

C:\Users\madhu\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

In [3]:
# Initialization of summarizer based on Bart
summarizer = pipeline("summarization", "vmarklynn/bart-large-cnn-samsum-acsi-ami-v2", truncation=True)
kw_model = keybert.KeyBERT(model='all-mpnet-base-v2')

C:\Users\madhu\AppData\Roaming\Python\Python311\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
def formatText(text):
    formatted_data = [f"{row['speaker_label']}: {row['text']}" for _, row in text.iterrows()]
    formatted_text = "\n".join([f"{line}" for line in formatted_data])
    return formatted_text

In [5]:
def summarizeText(transcirpt):
    
    
    text = formatText(transcirpt)
    
    #print("\n\n", text, "\n\n")
    # print( "min: ", math.ceil(int(wordCount) * 0.1), "max: ", math.ceil(int(wordCount) * 0.25))
    print("\n\nSummarizing...")
    summary = summarizer(text)[0]['summary_text']
    print("\n", summary, "\n")
    
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(1, 1), 
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)
    keywords_list_1= list(dict(keywords).keys())
    print(keywords_list_1)
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(2, 2), 
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)
    keywords_list_2= list(dict(keywords).keys())
    print(keywords_list_2)    
    keywords = kw_model.extract_keywords(text, 
                                     keyphrase_ngram_range=(3, 3), 
                                     stop_words='english', 
                                     highlight=False,
                                     top_n=5)    
    keywords_list_3 = list(dict(keywords).keys())
    print(keywords_list_3)
    
    response = {'transcription': formatText, 'summary': summary, 
                'keywords_list_1': keywords_list_1, 'keywords_list_2': keywords_list_2,
                'keywords_list_3': keywords_list_3,}
    return response

#-----------------------------------------------------------------------------------------------------               

def summarizeSummary(summary_input):
    
    
    wordCount = 1024 #post_data.get('wordCount-summ')
    
    print( "min: ", math.ceil(int(wordCount) * 0.1), "max: ", math.ceil(int(wordCount) * 0.25))
    print("\n\nSummarizing again...")
    summary = summarizer(summary_input, min_length = math.ceil(int(wordCount) * 0.1), max_length = math.ceil(int(wordCount) * 0.25))[0]['summary_text']
    print("\n", summary, "\n")
    
    response = {'summary': summary}
    return response

In [8]:
file_path='C:/Users/madhu/Desktop/SeattleUniversity/capstone/resonate_madhu/Resonate/Notebooks/Finance & Corporate Committee - Zoom Meeting.wav/Finance_Corporate_Committee.csv'
object_name = os.path.basename(file_path)

In [10]:
transcript=pd.read_csv(file_path)

In [11]:

transcript.head()
transcript.drop(['Unnamed: 0', 'end_time' ], axis=1, inplace=True)

In [12]:
import datetime

In [13]:
def convert_to_timestamp(time_field):
    microseconds = int(time_field * 1000000)  # Convert to microseconds
    time_stamp = datetime.datetime.utcfromtimestamp(microseconds / 1e6)  # Create datetime object
    return time_stamp

# Assuming 'transcript' is your DataFrame, apply the function to the 'start_time' column
transcript['start_time'] = transcript['start_time'].apply(convert_to_timestamp)

# Check if time exceeds 30 minutes


In [14]:
transcript.tail()


,start_time,speaker_label,text
2075,1970-01-01 00:01:43.777333,spk_0,"in a second. Thanks, Susan."
2076,1970-01-01 00:01:43.811483,spk_0,All in favor. I
2077,1970-01-01 00:01:43.842816,spk_0,I
2078,1970-01-01 00:01:43.863500,spk_0,can
2079,1970-01-01 00:01:43.872500,spk_0,"know carried. So Sam, we'll let you do your th..."


In [15]:
from datetime import datetime, timedelta
 # Convert 'timestamp' to datetime

# Find the index where the time exceeds 20 minutes
for idx, row in transcript.iterrows():
    if (row['start_time'].second >= 40):
        split_index = idx
        print(split_index)
        break
else:
    split_index = None

# Split DataFrame into two parts if split index is found
if split_index is not None:
    df1 = transcript.loc[:split_index].copy()  # Corrected to make copies
    df2 = transcript.loc[split_index + 1:].copy()  # Corrected to make copies

# Assuming your CSV file is named 'transcript.csv'




  


843


In [78]:
split_indices = []
found_multiples = set()  # Track found multiples of 20
for idx, row in transcript.iterrows():
    if row['start_time'].second % 20 == 0 and row['start_time'].second != 0:
        second_part = row['start_time'].second
        if second_part not in found_multiples:
            split_indices.append(idx)
            found_multiples.add(second_part)

# Split DataFrame into parts between consecutive split indices
df_parts = {}
prev_split_index = 0
for i, split_index in enumerate(split_indices):
    df_parts[f'df{i+1}'] = transcript.loc[prev_split_index:split_index].copy()
    prev_split_index = split_index + 1

# Include the part after the last split index
if prev_split_index < len(transcript):
    df_parts[f'df{i+2}'] = transcript.loc[prev_split_index:].copy()
    


In [79]:
print(split_indices)

[420, 874]


In [171]:
df_parts['df1']

,start_time,speaker_label,text,elapsed_seconds
0,1970-01-01 00:00:00.066000,spk_0,Right. Welcome everybody.,0.000000
1,1970-01-01 00:00:00.090000,spk_0,Um This is the first call about the um,0.024000
2,1970-01-01 00:00:00.170333,spk_0,um architecture design work for the effort tha...,0.104333
3,1970-01-01 00:00:00.246833,spk_0,the new diffs.,0.180833
4,1970-01-01 00:00:00.266316,spk_0,"Uh If anybody has any better naming, please su...",0.200316
...,...,...,...,...
416,1970-01-01 00:00:19.698333,spk_5,Is it like fully rolled out or do we have any ...,19.632333
417,1970-01-01 00:00:19.823833,spk_0,Z,19.757833
418,1970-01-01 00:00:19.827333,spk_0,do you want to talk about that?,19.761333
419,1970-01-01 00:00:19.934166,spk_6,Uh Are you referring to the syntax highlightin...,19.868166


In [172]:
df_parts['df2']

,start_time,speaker_label,text,elapsed_seconds
421,1970-01-01 00:00:20.017500,spk_6,Yeah.,19.951500
422,1970-01-01 00:00:20.034166,spk_6,Yeah. That would be a separate initiative. I t...,19.968166
423,1970-01-01 00:00:20.111333,spk_6,the syntax highlighting of the blobs,20.045333
424,1970-01-01 00:00:20.167816,spk_6,like the holidays that we use there doesn't re...,20.101816
425,1970-01-01 00:00:20.240166,spk_6,as far as I'm aware.,20.174166
...,...,...,...,...
870,1970-01-01 00:00:39.891666,spk_0,So we have seven minutes,39.825666
871,1970-01-01 00:00:39.920833,spk_0,um,39.854833
872,1970-01-01 00:00:39.952483,spk_0,"before I go to order priorities,",39.886483
873,1970-01-01 00:00:39.993666,spk_0,"I'll ask, should we have also an issue to disc...",39.927666


In [82]:
df_parts['df3']

,start_time,speaker_label,text,elapsed_seconds
875,1970-01-01 00:00:40.126666,spk_0,Should we have an issue to discuss this as syn...,40.060666
876,1970-01-01 00:00:40.166833,spk_0,with all with everyone?,40.100833
877,1970-01-01 00:00:40.183333,spk_0,Because I don't think we're gonna be making a ...,40.117333
878,1970-01-01 00:00:40.237500,spk_0,Um,40.171500
879,1970-01-01 00:00:40.252983,spk_0,We can take a step of an example of what that ...,40.186983
...,...,...,...,...
1129,1970-01-01 00:00:51.380333,spk_0,feel free to reach out to me and I can try to ...,51.314333
1130,1970-01-01 00:00:51.459000,spk_0,OK.,51.393000
1131,1970-01-01 00:00:51.472983,spk_0,That's it.,51.406983
1132,1970-01-01 00:00:51.483833,spk_0,We'll uh discuss the synchronously and we see ...,51.417833


In [17]:
df2.tail()

,start_time,speaker_label,text
2075,1970-01-01 00:01:43.777333,spk_0,"in a second. Thanks, Susan."
2076,1970-01-01 00:01:43.811483,spk_0,All in favor. I
2077,1970-01-01 00:01:43.842816,spk_0,I
2078,1970-01-01 00:01:43.863500,spk_0,can
2079,1970-01-01 00:01:43.872500,spk_0,"know carried. So Sam, we'll let you do your th..."


In [139]:
transcript.tail()

,start_time,speaker_label,text
733,1970-01-01 00:00:35.497983,spk_0,we can use the rest of the time to go into
734,1970-01-01 00:00:35.527316,spk_0,the issues and uh share our thoughts and make ...
735,1970-01-01 00:00:35.593666,spk_0,Go ahead.
736,1970-01-01 00:00:35.612150,spk_0,Uh Thanks everybody
737,1970-01-01 00:00:35.642166,spk_0,and uh thanks for your time and I'll see you a...


In [22]:
summary_transcript= summarizeText(df2)
print(summary_transcript['summary'])



Summarizing...

 The meeting was about the future of the airport company. Firstly, the group discussed the progress on the terminal redevelopment. They had a major land sale that was unconditional, but they had to pay a lot of attention to their balance sheet to keep up with the cost control. Next, they discussed the logistics of the construction of the central precinct. The group also discussed how they would make sure that the project was well-funded. Lastly, they talked about how the group would manage the costs of the project. 

['profitability', 'accommodation', 'financials', 'shareholders', 'resourcing']
['airport business', 'airport company', 'growth airport', 'financial performance', 'projects airport']
['growth airport business', 'diversified airport company', 'assist airport company', 'spk_5 airport company', 'airport business spk_6']
The meeting was about the future of the airport company. Firstly, the group discussed the progress on the terminal redevelopment. They had a 

In [23]:
summarized_summary = summarizeSummary(summary_transcript['summary'])

Your max_length is set to 256, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


min:  103 max:  256


Summarizing again...

 This meeting was about the future of the airport company. Firstly, the group discussed the progress on the terminal redevelopment. Next, they discussed the logistics of the construction of the central precinct. Lastly, they talked about how the group would manage the costs of the project. They had a major land sale that was unconditional, but they had to pay attention to their balance sheet to keep up with the cost control. The group also discussed how they would make sure that the project was well-funded.  



In [24]:
import pandas as pd
import os

def append_summary_to_csv(summary_text, cluster):
    # Define the CSV filename
    csv_filename = 'summary_new_data.csv'
    
    # Check if the CSV file exists
    if os.path.exists(csv_filename):
        # Load the existing data from the CSV file
        df = pd.read_csv(csv_filename)
    else:
        # Create a new DataFrame if the file doesn't exist
        df = pd.DataFrame(columns=['File_id','Text'])
    
    # Append the new summary and cluster data
    new_data = pd.DataFrame({'File_id':[object_name],'Text': [summary_text]})
    df = pd.concat([df, new_data], ignore_index=True)
    
    # Save the DataFrame to a CSV file
    df.to_csv(csv_filename, index=False)




In [25]:

summary_text = summarized_summary['summary']
cluster = "finance"


append_summary_to_csv(summary_text, cluster)